In [1]:
#导入需要使用的库
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import os
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.4f' % x) 
# pd.set_option('display.max_columns',50) #设置显示的最大列数，同时，也可以根据需要通过set_option函数设置其他的属性


## 载入数据

In [2]:
## 1) 载入训练集和测试集；
#remember to change the path to csv files each time before using this template
#path = 'C:/Users/87495/Desktop/Kaggle/predict-student-performance-from-game-play/'
path = 'D:/Code area/python area/kagglestuff/PredictStudentPerformanceFromGamePlay/'
Train_data = pd.read_csv(path+'train.csv', sep=',')
Test_data = pd.read_csv(path+'test.csv', sep=',')
Train_labels = pd.read_csv(path+'train_labels.csv', sep=',')
# Sampele_submission = pd.read_csv(path+'sample_submission.csv', sep=',')

# 通过调整数据类型，帮助我们减少数据在内存中占用的空间

In [3]:
#reduce_mem_usage 函数通过调整数据类型，帮助我们减少数据在内存中占用的空间
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
Train_data = reduce_mem_usage(Train_data)

Memory usage of dataframe is 2107873888.00 MB
Memory usage after optimization is: 816834322.00 MB
Decreased by 61.2%


In [5]:
Test_data = reduce_mem_usage(Test_data)

Memory usage of dataframe is 626432.00 MB
Memory usage after optimization is: 254036.00 MB
Decreased by 59.4%


In [6]:
# 合并方便后面的操作
df = pd.concat([Train_data, Test_data], ignore_index=True)

In [7]:
## 2) 简略观察数据(head()+shape)
Train_data.head().append(Train_data.tail())

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-414.0000,-159.3750,380.0000,494.0000,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-414.0000,-159.3750,380.0000,494.0000,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-414.0000,-159.3750,380.0000,494.0000,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-414.0000,-159.3750,380.0000,494.0000,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-413.0000,-159.3750,381.0000,494.0000,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
13174206,22100221145014656,1600,5483231,navigate_click,undefined,22,NaN,344.0000,36.6875,483.0000,273.0000,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
13174207,22100221145014656,1601,5485166,navigate_click,undefined,22,NaN,332.7500,141.5000,545.0000,221.0000,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
13174208,22100221145014656,1602,5485917,navigate_click,undefined,22,NaN,370.0000,140.6250,611.0000,217.0000,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
13174209,22100221145014656,1603,5486753,navigate_click,undefined,22,NaN,252.2500,123.8125,526.0000,232.0000,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
13174210,22100221145014656,1604,5487952,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [8]:
Train_data.shape

(13174211, 20)

In [9]:
Test_data.head().append(Train_data.tail())

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-414.0000,75.6875,380.0000,259.0000,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4,20090109393214576_0-4
1,20090109393214576,1,1965,person_click,basic,0,NaN,-106.0000,-63.3125,688.0000,398.0000,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4
2,20090109393214576,2,3614,person_click,basic,0,NaN,-419.0000,47.6875,375.0000,287.0000,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4
3,20090109393214576,3,5330,person_click,basic,0,NaN,-111.0000,-57.3125,683.0000,392.0000,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4
4,20090109393214576,4,6397,person_click,basic,0,NaN,-111.0000,-57.3125,683.0000,392.0000,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4
13174206,22100221145014656,1600,5483231,navigate_click,undefined,22,NaN,344.0000,36.6875,483.0000,273.0000,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22,NaN
13174207,22100221145014656,1601,5485166,navigate_click,undefined,22,NaN,332.7500,141.5000,545.0000,221.0000,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22,NaN
13174208,22100221145014656,1602,5485917,navigate_click,undefined,22,NaN,370.0000,140.6250,611.0000,217.0000,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22,NaN
13174209,22100221145014656,1603,5486753,navigate_click,undefined,22,NaN,252.2500,123.8125,526.0000,232.0000,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22,NaN
13174210,22100221145014656,1604,5487952,checkpoint,basic,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22,NaN


In [10]:
Test_data.shape

(3728, 21)

## 总览数据情况

In [11]:
#describe种有每列的统计量，个数count、平均值mean、方差std、最小值min、中位数25% 50% 75% 、以及最大值
#看这个信息主要是瞬间掌握数据的大概的范围以及每个值的异常值的判断，
#比如有的时候会发现999 9999 -1 等值这些其实都是nan的另外一种表达方式，有的时候需要注意下
#info 通过info来了解数据每列的type，有助于了解是否存在除了nan以外的特殊符号异常

In [12]:
## 1) 通过describe()来熟悉数据的相关统计量
Train_data.describe()

,session_id,index,elapsed_time,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fullscreen,hq,music
count,13174211.0000,13174211.0000,13174211.0000,13174211.0000,284746.0000,12137971.0000,12137971.0000,12137971.0000,12137971.0000,1000737.0000,0.0000,0.0000,0.0000
mean,21134134855458160.0000,652.6426,3846817.2860,12.1919,NaN,NaN,NaN,NaN,NaN,3186.2383,NaN,NaN,NaN
std,566522042999546.6250,627.5818,27013866.5246,6.4992,0.0000,NaN,NaN,NaN,NaN,369226.5312,NaN,NaN,NaN
min,20090312431273200.0000,0.0000,0.0000,0.0000,0.0000,-1992.0000,-918.0000,0.0000,0.0000,0.0000,NaN,NaN,NaN
25%,21010310162137956.0000,289.0000,439430.0000,6.0000,1.0000,-353.0000,-212.8750,269.0000,304.0000,100.0000,NaN,NaN,NaN
50%,21040215432620068.0000,596.0000,1013425.0000,13.0000,3.0000,-11.1641,-97.8125,447.0000,397.0000,418.0000,NaN,NaN,NaN
75%,21100514325903272.0000,897.0000,1740050.0000,18.0000,5.0000,296.2500,22.6875,663.0000,471.0000,1266.0000,NaN,NaN,NaN
max,22100221145014656.0000,20473.0000,1749293395.0000,22.0000,6.0000,1262.0000,543.5000,1916.0000,1439.0000,219907808.0000,NaN,NaN,NaN


In [13]:
## 2) 通过info()来熟悉数据类型
Train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13174211 entries, 0 to 13174210
Data columns (total 20 columns):
 #   Column          Dtype   
---  ------          -----   
 0   session_id      int64   
 1   index           int16   
 2   elapsed_time    int32   
 3   event_name      category
 4   name            category
 5   level           int8    
 6   page            float16 
 7   room_coor_x     float16 
 8   room_coor_y     float16 
 9   screen_coor_x   float16 
 10  screen_coor_y   float16 
 11  hover_duration  float32 
 12  text            category
 13  fqid            category
 14  room_fqid       category
 15  text_fqid       category
 16  fullscreen      float64 
 17  hq              float64 
 18  music           float64 
 19  level_group     category
dtypes: category(7), float16(5), float32(1), float64(3), int16(1), int32(1), int64(1), int8(1)
memory usage: 779.0 MB


## 转化数据类型

In [14]:
#astype
df.astype(int) 
##只能转化全部为数字组成的数据

#to_numeric
##pandas.to_numeric(arg, errors='raise', downcast=None)  
###默认返回dtype为float64或int64
###errors:{'ignore'，'raise'，'coerce'}，
# 默认为'raise'
# 如果为'raise',则无效的解析将引发异常;
# 如果为'coerce',则将无效解析设置为NaN;
# 如果为'ignore',则无效的解析将返回输入;
pd.to_numeric(df,errors='coerce')
##将字符串类型的数据转化为浮点型，并将原数据中由字母组成的字符串强制转化为NaN，但它是一个浮点数

ValueError: Cannot cast object dtype to int32

## 构造函数去除字符串中非数字的字符 

In [ ]:
def convert_money(value):
    new_value=value.replace("$","").replace(",","")
    return float(new_value)
df.apply(convert_money)
#非数字字符替换为空不是空格
convert_money1=lambda x: float(x.replace("%",""))
df.apply(convert_money1)
#lanbda函数（匿名函数）效果同上

## 将评价字段转换为数值

In [ ]:
def convert_pj(value):
    if "万" in value:
        new_value=float(value.replace("万",""))*10000
    else:
        new_value=value.replace("+","")
    return float(new_value)
data['评价']=data['评价'].astype(str).apply(convert_pj)
data['评价']

## 数据的一致性检查

In [ ]:
strlist=[]
for i in data['movie_name'].index:
    if isinstance(data['movie_name'][i],str)==False:
        strlist.append(i)
data['movie_name'][strlist]

## 处理重复值

In [ ]:
#检查重复值
df.duplicated(subset=[],keep='False')
#subset：指明数据子集，即某个特征或几个特征
# keep: 删除重复项并保留第一次出现的。取值可以为first last False
#first:保留第一个 last:保留最后一个 False:所有重复数据都标记为True
df.duplicated().any()#返回True，说明df有重复记录

In [ ]:
#删除重复值
df.drop_duplicates(keep='last',inplace=False)
#inplace=False生成一个删除了重复数据之后的新数据集，True会修改当前数据集

## 重复率

In [ ]:
data[data.duplicated()].count()/data.count()

## 处理缺失值

In [ ]:
#查看每列的存在nan情况
df.isnull().sum()

In [ ]:
# nan可视化
missing = Train_data.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()
#通过以上两句可以很直观的了解哪些列存在 “nan”, 并可以把nan的个数打印，主要的目的在于 nan存在的个数是否真的很大，
#如果很小一般选择填充，如果使用lgb等树模型可以直接空缺，让树自己去优化，但如果nan存在的过多、可以考虑删掉

In [ ]:
# 可视化看下缺省值
#msno库使用说明：https://blog.csdn.net/Andy_shenzl/article/details/81633356
import missingno as msno
msno.matrix(Train_data.sample(250))
msno.bar(Train_data.sample(1000))
msno.heatmap(Train_data.sample(250))
#missingno相关性热图措施无效的相关性：一个变量的存在或不存在如何强烈影响的另一个的存在：
    # 我们看到X5与X1.1的缺失相关性为1，说明X5只要发生了缺失，那么X1.1也会缺失，
    # X7和X8的相关性为-1，说明X7缺失的值，那么X8没有缺失；而X7没有缺失时，X8为缺失。

## 空缺率

In [ ]:
data_kql=(data.shape[0]-data.count())/data.count()
data_kql

# 进一步查看缺失特征中缺失率大于50%的特征

In [ ]:
have_null_fea_dict = (data_train.isnull().sum()/len(data_train)).to_dict()
fea_null_moreThanHalf = {}
for key,value in have_null_fea_dict.items():
    if value > 0.5:
        fea_null_moreThanHalf[key] = value

In [ ]:
fea_null_moreThanHalf

## 删除缺失值

In [ ]:
df.dropna()
'''
参数：
axis:       default 0指行,1为列
how:       {‘any’, ‘all’}, default ‘any’指带缺失值的所有行;'all’指清除全是缺失值的
thresh:    int,保留含有int个非空值的行
subset:   对特定的列进行缺失值删除处理
'''

## 填补缺失值

In [ ]:
df.fillna()

from sklearn.impute import SimpleImputer
df=SimpleImputer(*, missing_values=nan, strategy=‘mean’, fill_value=None, verbose=0, copy=True, add_indicator=False)

# 参数含义

# missing_values：int, float, str, (默认)np.nan或是None, 即缺失值是什么。

# strategy：空值填充的策略，共四种选择（默认）mean、median、most_frequent、constant。
#     mean表示该列的缺失值由该列的均值填充。
#     median为中位数，
#     most_frequent为众数。
#     constant表示将空值填充为自定义的值，但这个自定义的值要通过fill_value来定义。
    
# fill_value：str或数值，默认为Zone。
#     当strategy == "constant"时，fill_value被用来替换所有出现的缺失值（missing_values）。
#     fill_value为Zone，当处理的是数值数据时，缺失值（missing_values）会替换为0，对于字符串或对象数据类型则替换为"missing_value" 这一字符串。

# verbose：int，（默认）0，控制imputer的冗长。

# copy：boolean，（默认）True，表示对数据的副本进行处理，False对数据原地修改。

# add_indicator：boolean，（默认）False，True则会在数据后面加入n列由0和1构成的同样大小的数据，0表示所在位置非缺失值，1表示所在位置为缺失值。


## 根据规律填补（机器学习模型

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
df=pd.DataFrame({
    "one":np.random.randint(1,100,10),
    "two":[2,4,6,8,10,12,14,16,18,20],
    "three":[5,9,13,np.nan,21,np.nan,29,33,37,41]
})

df_train=df.dropna() #训练集
df_test=df[df['three'].isnull()]  #测试集,缺失值为需预测的值

regr=LinearRegression()
regr.fit(df_train['two'].values.reshape(-1,1),df_train['three'].values.reshape(-1,1))#训练出线性回归模型
df_three_pred=regr.predict(df_test['two'].values.reshape(-1,1))#用训练好的模型预测缺失值

#将所得数值填补到原数据集中
df.loc[(df.three.isnull()),'three']=df_three_pred
df
#df.loc[ 行索引, 列索引]


# z.reshape(-1)
# array([ 1,  2,  3,  4])
# z.reshape(-1,1)
#  array([[ 1],
#         [ 2],
#         [ 3],
#         [ 4]])
# 让z变成只有一列，行数不知道多少，通过`z.reshape(-1,1)`，Numpy自动计算

In [ ]:
df_test

## 删除异常值

In [ ]:
#以下两个类别特征严重倾斜，一般不会对预测有什么帮助，故这边先删掉，当然你也可以继续挖掘，但是一般意义不大
Train_data["seller"].value_counts()

In [ ]:
# 这里我包装了一个异常值处理的代码，可以随便调用。
def outliers_proc(data, col_name, scale=3):
    """
    用于清洗异常值，默认用 box_plot（scale=3）进行清洗
    :param data: 接收 pandas 数据格式
    :param col_name: pandas 列名
    :param scale: 尺度
    :return:
    """

    def box_plot_outliers(data_ser, box_scale):
        """
        利用箱线图去除异常值
        :param data_ser: 接收 pandas.Series 数据格式
        :param box_scale: 箱线图尺度，
        :return:
        """
        iqr = box_scale * (data_ser.quantile(0.75) - data_ser.quantile(0.25))
        val_low = data_ser.quantile(0.25) - iqr
        val_up = data_ser.quantile(0.75) + iqr
        rule_low = (data_ser < val_low)
        rule_up = (data_ser > val_up)
        return (rule_low, rule_up), (val_low, val_up)

    data_n = data.copy()
    data_series = data_n[col_name]
    rule, value = box_plot_outliers(data_series, box_scale=scale)
    index = np.arange(data_series.shape[0])[rule[0] | rule[1]]
    print("Delete number is: {}".format(len(index)))
    data_n = data_n.drop(index)
    data_n.reset_index(drop=True, inplace=True)
    print("Now column number is: {}".format(data_n.shape[0]))
    index_low = np.arange(data_series.shape[0])[rule[0]]
    outliers = data_series.iloc[index_low]
    print("Description of data less than the lower bound is:")
    print(pd.Series(outliers).describe())
    index_up = np.arange(data_series.shape[0])[rule[1]]
    outliers = data_series.iloc[index_up]
    print("Description of data larger than the upper bound is:")
    print(pd.Series(outliers).describe())
    
    fig, ax = plt.subplots(1, 2, figsize=(10, 7))
    sns.boxplot(y=data[col_name], data=data, palette="Set1", ax=ax[0])
    sns.boxplot(y=data_n[col_name], data=data_n, palette="Set1", ax=ax[1])
    return data_n

In [ ]:
# 我们可以删掉一些异常数据，以 power 为例。  
# 这里删不删同学可以自行判断
# 但是要注意 test 的数据不能删 = = 不能掩耳盗铃是不是

Train_data = outliers_proc(Train_data, 'power', scale=3)

## OneHot编码

In [ ]:
import pandas as pd
persons=pd.DataFrame({'name':['newton','andrew','jodn','bill'],'color':['yellow','white','black','white']})
print(persons)
df_dum=pd.get_dummies(persons['color'])
print(df_dum)
persons1=pd.concat([persons,df_dum],axis=1)
print(persons1)
#或者这一种简写
persons2 = pd.get_dummies(persons, columns=['color'])
print(persons2)

In [ ]:
pd.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)[source]
# data ： array-like，Series或DataFrame

# prefix ：string，字符串列表或字符串dict，默认为None，

# 用于追加DataFrame列名的字符串。在DataFrame上调用get_dummies时，传递一个长度等于列数的列表。或者，前缀 可以是将列名称映射到前缀的字典。

# prefix_sep ： string，默认为’_’

# 如果附加前缀，分隔符/分隔符要使用。或者传递与前缀一样的列表或字典。

# dummy_na ： bool，默认为False
# 如果忽略False NaN，则添加一列以指示NaN。

# columns ： 类似列表，默认为无
# 要编码的DataFrame中的列名称。如果列是None，那么所有与列 对象或类别 D型细胞将被转换。

# sparse ： bool，默认为False
# 伪编码列是否应由SparseArray（True）或常规NumPy数组（False）支持。

# drop_first ： bool，默认为False
# 是否通过删除第一级别从k分类级别获得k-1个假人。

print(pd.get_dummies(persons))
print(pd.get_dummies(persons,prefix='haha',prefix_sep='*'))


In [ ]:
persons=pd.concat([persons,df_dum],axis=1)
persons

## 数据离散化（分箱）

In [ ]:
#无监督离散化
ages=pd.DataFrame({'years':[10,140,30,53,67,32,45],'name':['a1','a2','a3','a4','a5','a6','a7']})
print(ages)
ages['label']=pd.cut(ages['years'],3)
print(ages)
ages['label']=pd.cut(ages['years'],3,labels=['青年','中年','老年']) #等宽分箱
print(ages)
ages['label']=pd.cut(ages['years'],bins=[9,20,50,200],labels=['青年','中年','老年'])  #bins不等宽分箱
print(ages)

#-------------------------------------------------------------------------

from sklearn.preprocessing import KBinsDiscretizer
kbd=KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='uniform')
#n_bins：表示所划分的区间个数（整数）
#encode：表示离散化后的结果保存方式
#     onehot：离散化后OnheHot编码，返回一个稀疏矩阵
#     onehot-dense：离散化后OnheHot编码，返回一个数组
#     ordinal:离散化后,以整数数值标记相应的记录
# strategy:离散化策略
#     uniform:分区的宽度相同
#     quantile:默认值,分区样本数量相同
#     kmeans:k-means聚类算法设置分区

ages=pd.DataFrame({'years':[10,140,30,53,67,32,45],'name':['a1','a2','a3','a4','a5','a6','a7']})
trans=kbd.fit_transform(ages[['years']])
ages['kbd']=trans[:,0]
print(ages)

# 了解预测值分布

In [ ]:
Train_data['price']

In [ ]:
Train_data['price'].value_counts()

In [ ]:
## 1) 总体分布概况（无界约翰逊分布等）
import scipy.stats as st
y = Train_data['price']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)
#价格不服从正态分布，所以在进行回归之前，它必须进行转换。虽然对数变换做得很好，但最佳拟合是无界约翰逊分布

In [ ]:
## 2) 查看skewness and kurtosis
sns.distplot(Train_data['price']);
print("Skewness: %f" % Train_data['price'].skew())
#数据的不对称程度 
#Skewness> 0,正偏差数值较大，为正偏或右偏。长尾巴拖在右边，数据右端有较多的极端值。
#Skewness < 0 ，负偏差数值较大，为负偏或左偏。长尾巴拖在左边，数据左端有较多的极端值。
#数值的绝对值越大，表明数据分布越不对称，偏斜程度大。
print("Kurtosis: %f" % Train_data['price'].kurt())
#数据分布顶的尖锐程度。
#（1）Kurtosis=0 与正态分布的陡缓程度相同。
#（2）Kurtosis>0 比正态分布的高峰更加陡峭——尖顶峰
#（3）Kurtosis<0 比正态分布的高峰来得平台——平顶峰

In [ ]:
Train_data.skew(), Train_data.kurt()

In [ ]:
sns.distplot(Train_data.skew(),color='blue',axlabel ='Skewness')

In [ ]:
sns.distplot(Train_data.kurt(),color='orange',axlabel ='Kurtness')
# skew、kurt说明参考https://www.cnblogs.com/wyy1480/p/10474046.html

In [ ]:
## 3) 查看预测值的具体频数
plt.hist(Train_data['price'], orientation = 'vertical',histtype = 'bar', color ='red')
plt.show()

#查看频数, 大于20000得值极少，其实这里也可以把这些当作特殊得值（异常值）直接用填充或者删掉，再前面进行
#orientation = 'vertical' 'horizontal',改变条形图方向
#查看预测值具体频数，右边有个较长的尾巴，用log变换来让图更符合正态

In [ ]:
# log变换 z之后的分布较均匀，可以进行log变换进行预测，这也是预测问题常用的trick
plt.hist(np.log(Train_data['price']), orientation = 'vertical',histtype = 'bar', color ='red') 
plt.show()

In [ ]:
#Box-Cox变换
    #Box-Cox变换不支持负值的输入；
    # Box-Cox变换对于lognormal and chi-squared分布（对数正态、卡方），表现好于Yeo-Johnson变换；
from sklearn.preprocessing import power_transform
dft2=power_transform(Train_data[['price']],method='box-cox')
plt.hist(dft2, orientation = 'vertical',histtype = 'bar', color ='red')

In [ ]:
#yeo-johnson变换
#Yeo-Johnson变换的特点在于其可被应用于包含0值和负值的样本中，因此其也被认为是Box-Cox变换在实数域的推广
from sklearn.preprocessing import power_transform
dft2=power_transform(Train_data[['price']],method='yeo-johnson')
plt.hist(dft2, orientation = 'vertical',histtype = 'bar', color ='red')

# 特征分为类别特征和数字特征，并对类别特征查看unique分布

In [ ]:
# 分离label即预测值
Y_train = Train_data['price']

In [ ]:
# 这个区别方式适用于没有直接label coding的数据
# 这里不适用，需要人为根据实际含义来区分
# 数字特征
# numeric_features = Train_data.select_dtypes(include=[np.number])
# numeric_features.columns
# 类型特征
categorical_features = Train_data.select_dtypes(include=[np.object])
categorical_features.columns



# numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
# category_fea = list(filter(lambda x: x not in numerical_fea,list(data_train.columns)))

In [ ]:
numeric_features = ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14' ]

categorical_features = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'regionCode',]

## 特征nunique分布

In [ ]:
# 特征nunique分布
#unique() 返回列的所有唯一值（特征的所有唯一值）
# nunique() 即返回的是唯一值的个数

for cat_fea in categorical_features:
    print(cat_fea + "的特征分布如下：")
    print("{}特征有个{}不同的值".format(cat_fea, Train_data[cat_fea].nunique()))
    print(Train_data[cat_fea].value_counts())

# 数字特征分析

In [ ]:
numeric_features.append('price')

In [ ]:
numeric_features

# 划分数值型变量中的连续变量和离散型变量

In [ ]:
#过滤数值型类别特征
def get_numerical_serial_fea(data,feas):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    for fea in feas:
        temp = data[fea].nunique()
        if temp <= 10:
            numerical_noserial_fea.append(fea)
            continue
        numerical_serial_fea.append(fea)
    return numerical_serial_fea,numerical_noserial_fea
numerical_serial_fea,numerical_noserial_fea = get_numerical_serial_fea(data_train,numerical_fea)

In [ ]:
numerical_serial_fea

In [ ]:
numerical_noserial_fea

In [ ]:
## 1) 相关性分析
price_numeric = Train_data[numeric_features]
correlation = price_numeric.corr()
print(correlation['price'].sort_values(ascending = False),'\n')
#数字特征分析是去做一个相关性的分析。主要是包括v开头的那些匿名特征，power，kilometer。需注意的是，相关性分析只对存在线性关系的变量有意义。

In [ ]:
f , ax = plt.subplots(figsize = (7, 7))
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.title('Correlation of Numeric Features with Price',y=1,size=16)
sns.set(font="simhei")
sns.heatmap(correlation,square = True,  vmax=0.8)

In [ ]:
del price_numeric['price']

In [ ]:
## 2) 查看几个特征得 偏度和峰值
for col in numeric_features:
    print('{:15}'.format(col), 
          'Skewness: {:05.2f}'.format(Train_data[col].skew()) , 
          '   ' ,
          'Kurtosis: {:06.2f}'.format(Train_data[col].kurt())  
         )

In [ ]:
## 3) 每个数字特征得分布可视化
f = pd.melt(Train_data, value_vars=numeric_features)
g = sns.FacetGrid(f, col="variable",  col_wrap=3, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")

## 可以看出匿名特征相对分布均匀

In [ ]:
## 4) 数字特征相互之间的关系可视化
sns.set()
columns = ['price', 'v_12', 'v_8' , 'v_0', 'power', 'v_5',  'v_2', 'v_6', 'v_1', 'v_14']
sns.pairplot(Train_data[columns],size = 2 ,kind ='scatter',diag_kind='kde')
plt.show()

In [ ]:
## 5) 多变量互相回归关系可视化
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(nrows=5, ncols=2, figsize=(24, 20))
# ['v_12', 'v_8' , 'v_0', 'power', 'v_5',  'v_2', 'v_6', 'v_1', 'v_14']
# Y_train:分离label即预测值
v_12_scatter_plot = pd.concat([Y_train,Train_data['v_12']],axis = 1)
sns.regplot(x='v_12',y = 'price', data = v_12_scatter_plot,scatter= True, fit_reg=True, ax=ax1)

v_8_scatter_plot = pd.concat([Y_train,Train_data['v_8']],axis = 1)
sns.regplot(x='v_8',y = 'price',data = v_8_scatter_plot,scatter= True, fit_reg=True, ax=ax2)

v_0_scatter_plot = pd.concat([Y_train,Train_data['v_0']],axis = 1)
sns.regplot(x='v_0',y = 'price',data = v_0_scatter_plot,scatter= True, fit_reg=True, ax=ax3)

power_scatter_plot = pd.concat([Y_train,Train_data['power']],axis = 1)
sns.regplot(x='power',y = 'price',data = power_scatter_plot,scatter= True, fit_reg=True, ax=ax4)

v_5_scatter_plot = pd.concat([Y_train,Train_data['v_5']],axis = 1)
sns.regplot(x='v_5',y = 'price',data = v_5_scatter_plot,scatter= True, fit_reg=True, ax=ax5)

v_2_scatter_plot = pd.concat([Y_train,Train_data['v_2']],axis = 1)
sns.regplot(x='v_2',y = 'price',data = v_2_scatter_plot,scatter= True, fit_reg=True, ax=ax6)

v_6_scatter_plot = pd.concat([Y_train,Train_data['v_6']],axis = 1)
sns.regplot(x='v_6',y = 'price',data = v_6_scatter_plot,scatter= True, fit_reg=True, ax=ax7)

v_1_scatter_plot = pd.concat([Y_train,Train_data['v_1']],axis = 1)
sns.regplot(x='v_1',y = 'price',data = v_1_scatter_plot,scatter= True, fit_reg=True, ax=ax8)

v_14_scatter_plot = pd.concat([Y_train,Train_data['v_14']],axis = 1)
sns.regplot(x='v_14',y = 'price',data = v_14_scatter_plot,scatter= True, fit_reg=True, ax=ax9)

v_13_scatter_plot = pd.concat([Y_train,Train_data['v_13']],axis = 1)
sns.regplot(x='v_13',y = 'price',data = v_13_scatter_plot,scatter= True, fit_reg=True, ax=ax10)


# 类别特征分析

In [ ]:
## 1) unique分布
for fea in categorical_features:
    print(Train_data[fea].nunique())
    
# 对于一维数组或者列表，unique函数去除其中重复的元素，并按元素由大到小返回一个新的无元素重复的元组或者列表。

In [ ]:
categorical_features

In [ ]:
## 2) 类别特征箱形图可视化

# 因为 name和 regionCode的类别太稀疏了，这里我们把不稀疏的几类画一下
categorical_features = ['model',
 'brand',
 'bodyType',
 'fuelType',
 'gearbox',
 'notRepairedDamage']
for c in categorical_features:
    Train_data[c] = Train_data[c].astype('category')
    if Train_data[c].isnull().any():
        Train_data[c] = Train_data[c].cat.add_categories(['MISSING'])
        Train_data[c] = Train_data[c].fillna('MISSING')

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

f = pd.melt(Train_data, id_vars=['price'], value_vars=categorical_features)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "price")

In [ ]:
Train_data.columns

In [ ]:
## 3) 类别特征的小提琴图可视化
catg_list = categorical_features
target = 'price'
for catg in catg_list :
    sns.violinplot(x=catg, y=target, data=Train_data)
    plt.show()

In [ ]:
categorical_features = ['model',
 'brand',
 'bodyType',
 'fuelType',
 'gearbox',
 'notRepairedDamage']

In [ ]:
## 4) 类别特征的柱形图可视化
def bar_plot(x, y, **kwargs):
    sns.barplot(x=x, y=y)
    x=plt.xticks(rotation=90)

f = pd.melt(Train_data, id_vars=['price'], value_vars=categorical_features)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(bar_plot, "value", "price")

In [ ]:
##  5) 类别特征的每个类别频数可视化(count_plot)
def count_plot(x,  **kwargs):
    sns.countplot(x=x)
    x=plt.xticks(rotation=90)

f = pd.melt(Train_data,  value_vars=categorical_features)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(count_plot, "value")

# 用pandas_profiling生成数据报告

In [ ]:
#用pandas_profiling生成一个较为全面的可视化和数据报告(较为简单、方便) 最终打开html文件即可
import pandas_profiling

In [ ]:
pfr = pandas_profiling.ProfileReport(Train_data)
pfr.to_file("./example.html")

In [ ]:
# 所给出的EDA步骤为广为普遍的步骤，在实际的不管是工程还是比赛过程中，这只是最开始的一步，也是最基本的一步。
# 接下来一般要结合模型的效果以及特征工程等来分析数据的实际建模情况，根据自己的一些理解，查阅文献，对实际问题做出判断和深入的理解。
# 最后不断进行EDA与数据处理和挖掘，来到达更好的数据结构和分布以及较为强势相关的特征
# 数据探索在机器学习中我们一般称为EDA（Exploratory Data Analysis）：
# 是指对已有的数据（特别是调查或观察得来的原始数据）在尽量少的先验假定下进行探索，通过作图、制表、方程拟合、计算特征量等手段探索数据的结构和规律的一种数据分析方法。
# 数据探索有利于我们发现数据的一些特性，数据之间的关联性，对于后续的特征构建是很有帮助的。
# 对于数据的初步分析（直接查看数据，或.sum(), .mean()，.descirbe()等统计函数）可以从：样本数量，训练集数量，是否有时间特征，是否是时许问题，特征所表示的含义（非匿名特征），特征类型（字符类似，int，float，time），特征的缺失情况（注意缺失的在数据中的表现形式，有些是空的有些是”NAN”符号等），特征的均值方差情况。
# 分析记录某些特征值缺失占比30%以上样本的缺失处理，有助于后续的模型验证和调节，分析特征应该是填充（填充方式是什么，均值填充，0填充，众数填充等），还是舍去，还是先做样本分类用不同的特征模型去预测。
# 对于异常值做专门的分析，分析特征异常的label是否为异常值（或者偏离均值较远或者事特殊符号）,异常值是否应该剔除，还是用正常值填充，是记录异常，还是机器本身异常等。
# 对于Label做专门的分析，分析标签的分布情况等。
# 进步分析可以通过对特征作图，特征和label联合做图（统计图，离散图），直观了解特征的分布情况，通过这一步也可以发现数据之中的一些异常值等，通过箱型图分析一些特征值的偏离情况，对于特征和特征联合作图，对于特征和label联合作图，分析其中的一些关联性。

In [ ]:
#循环特征选择
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

data=pd.read_csv('.csv')
x=data.iloc[:,0:14] #前十四个字段做自变量
y=data.iloc[:,-1].values #最后一个做应变量
knn=KNeighborsClassifier(n_neighbors=4)
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1=SFS(knn,k_features=4,forward=True,floating=False,verbose=2,scoring='accuracy',cv=0)
sfs1=sfs1.fit(x,y)
sfs1.subset_

